### Финальная работа 

В рамках проекта я решила проанализировать свой телеграмм канал. Было решено провести сентимент-анализ: использовать модель машинного обучения для определения тональности сообщений (негативная, нейтральная, положительная). 

Также стало интересно проанализировать чат канала чтобы узнать самых активных участников.


In [1]:
import pandas as pd
from telethon.sync import TelegramClient

In [28]:
api_id = '-'
api_hash = '-'
phone = '-'

In [ ]:
client=TelegramClient(phone, api_id, api_hash)
await client.start()
print(await client.get_me())

Signed in successfully as sussy; remember to not break the ToS or you will risk an account ban!
User(id=746665928, is_self=True, contact=True, mutual_contact=True, deleted=False, bot=False, bot_chat_history=False, bot_nochats=False, verified=False, restricted=False, min=False, bot_inline_geo=False, support=False, scam=False, apply_min_photo=True, fake=False, bot_attach_menu=False, premium=True, attach_menu_enabled=False, bot_can_edit=False, close_friend=False, stories_hidden=False, stories_unavailable=True, contact_require_premium=False, bot_business=False, access_hash=2198029915899487390, first_name='sussy', last_name=None, username='rechenie', phone='79261720590', photo=UserProfilePhoto(photo_id=3206905742253729788, dc_id=2, has_video=False, personal=False, stripped_thumb=b'\x01\x08\x08\xcfhc\x16\xbb\xc3\x1d\xf9\xe9E\x14R\x1bG'), status=UserStatusOnline(expires=datetime.datetime(2024, 5, 31, 15, 36, 58, tzinfo=datetime.timezone.utc)), bot_info_version=None, restriction_reason=[], bot

Server sent a very old message with ID 7375190760691146753, ignoring (see FAQ for details)
Server sent a very old message with ID 7375190768629932033, ignoring (see FAQ for details)
Server sent a very old message with ID 7375190785993061385, ignoring (see FAQ for details)
Server sent a very old message with ID 7375190785993061381, ignoring (see FAQ for details)
Server sent a very old message with ID 7375190785993061389, ignoring (see FAQ for details)
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server closed the connection: [Errno 54] Connection reset by peer
Server sent a ver

In [61]:
chat = 'https://t.me/notrechenie'

Сначала считаем первые 3000 новых постов

In [129]:
data_message_new = []
part_mes = await client.get_messages(chat, limit=3000)

for message in part_mes:
  data_message_new.append([message.sender_id, message.text])

In [134]:
df_new = pd.DataFrame(data_message_new, columns=['user_id', 'text'])
df_new

,user_id,text
0,-1001871224871,И ТАК БУДЕТ С КАЖДЫМ \nжду вас в нашей кофейне 🤗
1,-1001871224871,по правилам нам надо извиниться и переделать\n...
2,-1001871224871,"я уже настолько привыкла к этому, что запомина..."
3,-1001871224871,я в шоке типа\nвы думаете я дура?\nесли ты про...
4,-1001871224871,- а кофе без сахара чтоли?\n- вы не просили са...
...,...,...
2995,-1001871224871,получается можно вообще не пользоваться шампун...
2996,-1001871224871,еду домой.\nкстати как видите волосы как были ...
2997,-1001871224871,если что это реально трусы. я рассмотрела дета...
2998,-1001871224871,чьи трусы…


In [136]:
df_new.to_csv('messages_new.csv')
df_new.shape

(3000, 2)

In [137]:
df_new=pd.read_csv('messages_new.csv', sep=',')
df_new.shape

(3000, 3)

Удалим строки с пустыми значениями

In [140]:
df_new=df_new.dropna(axis=0, how='any')
df_new.shape

(2485, 3)

Теперь первые 3000 постов

In [141]:
data_message_old = []
part_mes= await client.get_messages(chat, limit=3000, reverse=True)

for message in part_mes:
  data_message_old.append([message.sender_id, message.text])

In [142]:
df_old = pd.DataFrame(data_message_old, columns=['user_id', 'text'])
df_old

,user_id,text
0,-1001871224871,None
1,-1001871224871,а как сделать комментарии
2,-1001871224871,ХАХААХ кто посмотрел время 6 утра 💀
3,-1001871224871,спасибо за сердечко 🫶
4,-1001871224871,а вы же поняли что я сусанна? \nвдруг не поняли 😔
...,...,...
2995,-1001871224871,"люди которые шарят за новость, теперь операции..."
2996,-1001871224871,я хочу быть умной чтобы в разговоре про матку ...
2997,-1001871224871,ПРИЗНАЮ я люблю журналистику\nлюблю журналисто...
2998,-1001871224871,"нина, которая делает вид, что не разбирается в..."


In [143]:
df_old.to_csv('messages_old.csv')
df_old.shape

(3000, 2)

In [144]:
df_old=pd.read_csv('messages_old.csv', sep=',')
df_old.shape

(3000, 3)

Также удалим строки с пустыми значениями

In [145]:
df_old=df_old.dropna(axis=0, how='any')
df_old.shape

(2525, 3)

# Семантический анализ

In [12]:
from transformers import pipeline

2024-05-31 18:40:05.689183: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
sentiment_analysis = pipeline("sentiment-analysis", model="blanchefort/rubert-base-cased-sentiment")

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [165]:
df_model_new = df_new.copy()
lst = []
for i in df_model_new["text"]:
  lst.append(sentiment_analysis(str(i))[0]["label"])
df_model_new["Sentinent"]=pd.DataFrame(lst)
df_model_new

,Unnamed: 0,user_id,text,Sentinent
0,0,-1001871224871,И ТАК БУДЕТ С КАЖДЫМ \nжду вас в нашей кофейне 🤗,POSITIVE
1,1,-1001871224871,по правилам нам надо извиниться и переделать\n...,NEGATIVE
2,2,-1001871224871,"я уже настолько привыкла к этому, что запомина...",NEGATIVE
3,3,-1001871224871,я в шоке типа\nвы думаете я дура?\nесли ты про...,NEGATIVE
4,4,-1001871224871,- а кофе без сахара чтоли?\n- вы не просили са...,POSITIVE
...,...,...,...,...
2995,2995,-1001871224871,получается можно вообще не пользоваться шампун...,NaN
2996,2996,-1001871224871,еду домой.\nкстати как видите волосы как были ...,NaN
2997,2997,-1001871224871,если что это реально трусы. я рассмотрела дета...,NaN
2998,2998,-1001871224871,чьи трусы…,NaN


In [166]:
df_model_new_cleaned = df_model_new.dropna(axis=0, how='any')
df_model_new_cleaned

,Unnamed: 0,user_id,text,Sentinent
0,0,-1001871224871,И ТАК БУДЕТ С КАЖДЫМ \nжду вас в нашей кофейне 🤗,POSITIVE
1,1,-1001871224871,по правилам нам надо извиниться и переделать\n...,NEGATIVE
2,2,-1001871224871,"я уже настолько привыкла к этому, что запомина...",NEGATIVE
3,3,-1001871224871,я в шоке типа\nвы думаете я дура?\nесли ты про...,NEGATIVE
4,4,-1001871224871,- а кофе без сахара чтоли?\n- вы не просили са...,POSITIVE
...,...,...,...,...
2480,2480,-1001871224871,полина скинула мне плохо,NEUTRAL
2481,2481,-1001871224871,ок,NEGATIVE
2482,2482,-1001871224871,"девочки, с праздником❤️❤️",NEUTRAL
2483,2483,-1001871224871,краткость - сестра таланта \nвсе сходится\nсте...,NEUTRAL


In [167]:
# Подсчет количества позитивных и негативных постов
positive_count = df_model_new_cleaned[df_model_new_cleaned['Sentinent'] == 'POSITIVE'].shape[0]
negative_count = df_model_new_cleaned[df_model_new_cleaned['Sentinent'] == 'NEGATIVE'].shape[0]

# Вывод результатов
print(f"Количество позитивных постов: {positive_count}")
print(f"Количество негативных постов: {negative_count}")

Количество позитивных постов: 387
Количество негативных постов: 732


In [168]:
df_model_old = df_old.copy()
lst = []
for i in df_model_old["text"]:
  lst.append(sentiment_analysis(str(i))[0]["label"])
df_model_old["Sentinent"]=pd.DataFrame(lst)
df_model_old

,Unnamed: 0,user_id,text,Sentinent
1,1,-1001871224871,а как сделать комментарии,POSITIVE
2,2,-1001871224871,ХАХААХ кто посмотрел время 6 утра 💀,POSITIVE
3,3,-1001871224871,спасибо за сердечко 🫶,POSITIVE
4,4,-1001871224871,а вы же поняли что я сусанна? \nвдруг не поняли 😔,POSITIVE
5,5,-1001871224871,объясняю почему вы здесь!\n\nэто значит у вас ...,POSITIVE
...,...,...,...,...
2995,2995,-1001871224871,"люди которые шарят за новость, теперь операции...",NaN
2996,2996,-1001871224871,я хочу быть умной чтобы в разговоре про матку ...,NaN
2997,2997,-1001871224871,ПРИЗНАЮ я люблю журналистику\nлюблю журналисто...,NaN
2998,2998,-1001871224871,"нина, которая делает вид, что не разбирается в...",NaN


In [169]:
df_model_old_cleaned = df_model_old.dropna(axis=0, how='any')
df_model_old_cleaned

,Unnamed: 0,user_id,text,Sentinent
1,1,-1001871224871,а как сделать комментарии,POSITIVE
2,2,-1001871224871,ХАХААХ кто посмотрел время 6 утра 💀,POSITIVE
3,3,-1001871224871,спасибо за сердечко 🫶,POSITIVE
4,4,-1001871224871,а вы же поняли что я сусанна? \nвдруг не поняли 😔,POSITIVE
5,5,-1001871224871,объясняю почему вы здесь!\n\nэто значит у вас ...,POSITIVE
...,...,...,...,...
2520,2520,-1001871224871,АХАХАХАХА 😀,NEGATIVE
2521,2521,-1001871224871,🤣,NEUTRAL
2522,2522,-1001871224871,боже подъезд токсичных кринжей нет слов,POSITIVE
2523,2523,-1001871224871,я прошла игру в роблоксе,POSITIVE


In [170]:
# Подсчет количества позитивных и негативных постов
positive_count = df_model_old_cleaned[df_model_old_cleaned['Sentinent'] == 'POSITIVE'].shape[0]
negative_count = df_model_old_cleaned[df_model_old_cleaned['Sentinent'] == 'NEGATIVE'].shape[0]

# Вывод результатов
print(f"Количество позитивных постов: {positive_count}")
print(f"Количество негативных постов: {negative_count}")

Количество позитивных постов: 602
Количество негативных постов: 605


# Анализ чата канала

In [ ]:
client=TelegramClient(phone, api_id, api_hash)
await client.start()
print(await client.get_me())

In [147]:
chat2 = 'https://t.me/chat_notrechenie'

In [148]:
data_message_chat = []
part_mes= await client.get_messages(chat2, limit=3000)

for message in part_mes:
  data_message_chat.append([message.sender_id, message.text])

In [158]:
df_chat = pd.DataFrame(data_message_chat, columns=['user_id', 'text'])
df_chat.to_csv('messages_chat.csv')
df_chat

,user_id,text
0,1.073224e+09,мили
1,1.016946e+09,я хз как это работает но у нее эта хуйня появи...
2,1.016946e+09,кстати мне коллега сказала что это база
3,-1.001871e+12,И ТАК БУДЕТ С КАЖДЫМ \nжду вас в нашей кофейне 🤗
4,8.627878e+08,хаххахха туда его
...,...,...
2995,NaN,за мой счет
2996,NaN,тебе бесплатно сделаю
2997,1.017542e+09,и сколько там стоит кофе
2998,NaN,


In [159]:
df_chat=pd.read_csv('messages_chat.csv')
df_chat.shape

(3000, 3)

In [160]:
df_chat=df_chat.dropna(axis=0, how='any')
df_chat.shape

(2041, 3)

# Нахождение самых активных пользователей

Выведем id всех людей оставлявших комментарии

In [161]:
user_message_count = df_chat['user_id'].value_counts()
user_message_count

user_id
-1.001871e+12    730
 9.925694e+08    241
 1.017542e+09    237
 1.834487e+09    157
 1.073224e+09    114
 7.850631e+08    109
 3.785969e+08     68
 3.267158e+08     51
 1.016946e+09     46
 1.059864e+09     42
 6.843616e+09     42
 3.584742e+08     40
 2.628276e+08     26
 1.171426e+09     22
 8.627878e+08     20
 8.319387e+08     19
 7.434573e+08     18
 7.468028e+08     13
 5.702539e+08     13
 4.953622e+08      9
 1.282894e+09      7
 8.610990e+08      5
 9.221176e+08      4
 9.567740e+08      2
 3.802541e+08      2
 3.796020e+08      2
 4.640574e+08      1
 6.037067e+08      1
Name: count, dtype: int64

Создадим словарь с id и количеством сообщений

In [162]:
# user_ids = user_message_count.head(10).index.tolist()
# user_ids
id_count_dict = user_message_count.to_dict()
id_count_dict

{-1001871224871.0: 730,
 992569407.0: 241,
 1017541933.0: 237,
 1834486913.0: 157,
 1073224408.0: 114,
 785063118.0: 109,
 378596903.0: 68,
 326715753.0: 51,
 1016946174.0: 46,
 1059863541.0: 42,
 6843615884.0: 42,
 358474242.0: 40,
 262827643.0: 26,
 1171425897.0: 22,
 862787766.0: 20,
 831938692.0: 19,
 743457334.0: 18,
 746802816.0: 13,
 570253865.0: 13,
 495362205.0: 9,
 1282894199.0: 7,
 861098954.0: 5,
 922117640.0: 4,
 956773981.0: 2,
 380254144.0: 2,
 379602042.0: 2,
 464057434.0: 1,
 603706685.0: 1}

Теперь выведем подписчиков в порядке убывания количества их комментариев

In [164]:
data_item = []
 
part_item = await client.get_participants(chat)
for item in part_item:
  if item.id in id_count_dict.keys():
    data_item.append([item.first_name, item.id, id_count_dict[item.id]])

df_names = pd.DataFrame(data_item, columns=['name', 'id', 'count'])
df_names_sorted = df_names.sort_values('count', ascending=False)
df_names_sorted

,name,id,count
8,нинок хачапуридзе,992569407,241
4,ибра gradop,1017541933,237
9,женька,1834486913,157
12,таня!,1073224408,114
22,мария ♨️,785063118,109
16,фидан,378596903,68
6,елена,326715753,51
7,˗ˏˋоля,1016946174,46
5,Julia,6843615884,42
10,(｡•́︿•̀｡) катя нины,1059863541,42
